In [1]:
import nltk
from nltk.corpus import names
from pylab import *
import random as pyrandom

# Text Classification

Let's start with a very simple text classification problem: guessing the gender of a name from the name itself.

You can probably make a pretty good guess about the gender of names like: "Bilama" or "Telek".

If we want to generalize to new names, we need to extract properties of names that occur in new, previously unseen names. We call these properties _features_.

In NLTK, they are represented as hash tables.

In [2]:
def gender_features(w):
    return dict(last_letter=w[-1],
                first_letter=w[0],
                length=len(w))
gender_features("Petra")

{'first_letter': 'P', 'last_letter': 'a', 'length': 5}

For the training data, we read male and female names from NLTK corpora.

In [3]:
male = [(name,'male') for name in names.words('male.txt')]
female = [(name,'female') for name in names.words('female.txt')]
nlist = male+female
print len(nlist),len(set([x for x,y in nlist]))

7944 7579


In [4]:
pyrandom.shuffle(nlist)
nlist[:5]

[('Pier', 'female'),
 ('Laird', 'male'),
 ('Ramsay', 'male'),
 ('Wilburt', 'male'),
 ('Roy', 'male')]

We extract features and then split the data into a training set and a test set.

In [5]:
featuresets = [(gender_features(n),g) for n,g in nlist]
training_set = featuresets[500:]
test_set = featuresets[:500]

In [6]:
featuresets[:5]

[({'first_letter': 'P', 'last_letter': 'r', 'length': 4}, 'female'),
 ({'first_letter': 'L', 'last_letter': 'd', 'length': 5}, 'male'),
 ({'first_letter': 'R', 'last_letter': 'y', 'length': 6}, 'male'),
 ({'first_letter': 'W', 'last_letter': 't', 'length': 7}, 'male'),
 ({'first_letter': 'R', 'last_letter': 'y', 'length': 3}, 'male')]

Once we have features and corresponding labels, we can train a classifier...

In [7]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

... and evaluate its performance on the test set.

In [8]:
nltk.classify.accuracy(classifier,test_set)

0.788

Classiifers also give us information about how informative features are.

In [9]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     35.9 : 1.0
             last_letter = 'k'              male : female =     32.7 : 1.0
             last_letter = 'f'              male : female =     15.9 : 1.0
             last_letter = 'p'              male : female =     12.6 : 1.0
             last_letter = 'v'              male : female =     10.5 : 1.0


Naive Bayesian classifiers assume a very simple statistical model of the posterior probability $P(c|x)$ for input features $x = (x_1,...,x_n)$

- We assume that each feature is generated independently $P(x|c) = \prod_i(x_i|c)$
- We use Bayes formula to turn that equation into a posterior probability $P(c|x)$

Here, the different $P(x_i|c)$ are modeled via empirical distributions; that is, we count
how often $x_i$ is true given that the class is $c$.

# Another Classifier

There are many different classifiers available in NLTK; they give different performance on different
tasks. There is no single best classifier, so you need a bit of experimentation.

In [10]:
classifier = nltk.MaxentClassifier.train(training_set,algorithm="IIS",max_iter=10)
nltk.classify.accuracy(classifier,test_set)

  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.370
             2          -0.47845        0.733
             3          -0.42183        0.774
             4          -0.39399        0.780
             5          -0.37861        0.783
             6          -0.36935        0.783
             7          -0.36341        0.782
             8          -0.35944        0.782
             9          -0.35671        0.781
         Final          -0.35477        0.781


0.798

The maximum entropy classifier can be derived in different ways. In practice, it amounts to the same classifier as logistic regression:

$P(c|x) = \sigma(w \cdot x)$

where $\sigma(x) = \frac{1}{1+e^{-x}}$

The term "MaxEnt" is frequently used in NLP. It refers to the fact that we are thinking of the problem as follows:

- assume that we have a set of binary feature functions of documents $x_i(d)$
- each binary feature function is true or false
- for each binary feature function, we have a posterior $P(c|x_i)$
- we want to find an overall posterior probability $P(c|d)$ that is...
  - consistent with the individual posteriors
  - otherwise a "maximum entropy distribution"

# "Traditional" Classifier

The NLTK classifiers take features in the form of hash tables; this is convenient for NLP tasks, but somewhat inefficient.

Classifiers in other machine learning libraries tend to take input data in a different format.

A common format is two matrices, one for inputs (each row representing an input vectors), and one for outputs (containing integer classes or indicator functions).

In [11]:
xs = zeros((len(training_set),26))
ys = zeros(len(training_set))

For coding the inputs, we use a "unary code".

In [12]:
for i,(f,c) in enumerate(training_set):
    ll = f["last_letter"].lower()
    if ll==" " : continue
    xs[i,ord(ll)-ord("a")] = 1
    if c=="female": ys[i] = 1

*LogisticRegression* is the same as *MaxentClassifier*, but the sklearn implementation is much faster.

In [13]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(xs,ys)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', tol=0.0001)

In [14]:
xs = zeros((len(test_set),26))
ys = zeros(len(test_set))
for i,(f,c) in enumerate(test_set):
    ll = f["last_letter"].lower()
    if ll==" " : continue
    xs[i,ord(ll)-ord("a")] = 1
    if c=="female": ys[i] = 1

In [15]:
1.0-sum(lr.predict(xs)!=ys)*1.0/len(ys)

0.754

# Bigger Feature Set

There is no single right feature set, and different feature sets give different amounts of performance for different classifiers.

In [16]:
def more_features(w):
    features = {}
    features["first"] = w[0].lower()
    features["last"] = w[-1].lower()
    features["last2"] = w[-2:].lower()
    for c in [chr(i) for i in range(ord("a"),ord("z")+1)]:
        features["nr_"+c] = name.lower().count(c)
        features["has_"+c] = (c in name.lower())
    return features

In [17]:
featuresets = [(more_features(n),g) for n,g in nlist]
training_set = featuresets[500:]
test_set = featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(training_set)
nltk.classify.accuracy(classifier,test_set)

0.786

Usually, you should split the training data into three sets:

- the training set
- the feature evaluation set
- the test set

If you don't, you risk that you get a good result on the test set by accident, a result that doesn't generalize.

Other approaches are resampling methods and cross-validation.

What are some of the tradeoffs in choosing a feature set?

# Decision Trees

Decision trees are another common classifier.

In [18]:
def simple_features(w):
    return {'fl':w[0].lower(),'ll': w[-1].lower(),'l':len(w)}
simple_features("Petra")

{'fl': 'p', 'l': 5, 'll': 'a'}

In [19]:
featuresets = [(simple_features(n),g) for n,g in nlist]
training_set = featuresets[500:]
test_set = featuresets[:500]

In [20]:
classifier = nltk.DecisionTreeClassifier.train(training_set,depth_cutoff=2)
nltk.classify.accuracy(classifier,test_set)

0.77

In [21]:
print classifier.pseudocode()

if ll == 'a': return 'female'
if ll == 'b': 
  if fl == 'a': return 'male'
  if fl == 'b': return 'female'
  if fl == 'c': return 'male'
  if fl == 'd': return 'female'
  if fl == 'h': return 'male'
  if fl == 'j': return 'male'
  if fl == 'k': return 'male'
  if fl == 'l': return 'female'
  if fl == 'm': return 'female'
  if fl == 'r': return 'male'
  if fl == 't': return 'male'
  if fl == 'w': return 'male'
  if fl == 'z': return 'male'
if ll == 'c': return 'male'
if ll == 'd': return 'male'
if ll == 'e': 
  if fl == 'a': return 'female'
  if fl == 'b': return 'female'
  if fl == 'c': return 'female'
  if fl == 'd': return 'female'
  if fl == 'e': return 'female'
  if fl == 'f': return 'female'
  if fl == 'g': return 'female'
  if fl == 'h': return 'female'
  if fl == 'i': return 'female'
  if fl == 'j': return 'female'
  if fl == 'k': return 'female'
  if fl == 'l': return 'female'
  if fl == 'm': return 'female'
  if fl == 'n': return 'female'
  if fl == 'o': return 'female'
  if f

Decision trees are classifiers that classify as a nested sequence of if-then statements.

Variables can be binary, categorical, or numeric.

For numerical variables, they divide the feature space into axis-parallel rectangles and associated probabilities.

Decision trees are generally grown as follows:

- take a set of data
- consider splits along every possible feature and value
- pick the best split according to the minimal impurity of the corresponding label set
- split according to that feature and value
- repeat the process on each subset (branch)
- stop if a minimum impurity or set size is reached

A better way of doing this is to split like the above, into small terminal nodes (deliberate overfitting), then start merging terminal nodes back together again, based on cross-validated error ("pruning"). This is what CART does, and leads to better overall performance.

# Document Classification

In [22]:
from nltk.corpus import movie_reviews

In [23]:
documents = [(list(movie_reviews.words(fileid)),category)
                 for category in movie_reviews.categories()
                     for fileid in movie_reviews.fileids(category)]

In [24]:
pyrandom.shuffle(documents)

In [25]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())

In [26]:
word_features = all_words.keys()[:2000]

In [27]:
def document_features(document):
    document_words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in document_words)
    return features

In [28]:
print document_features(documents[0][0])

{'limited': False, 'four': False, 'woods': True, 'woody': False, 'captain': False, 'hate': False, 'consider': False, 'relationships': False, 'whose': False, 'buddy': False, 'themes': False, 'presents': False, 'edward': False, 'under': False, 'lord': False, 'worth': False, 'rescue': False, 'every': False, 'jack': False, 'bringing': False, 'school': False, 'skills': False, 'ups': False, 'enjoy': False, 'force': False, 'tired': False, 'miller': False, 'direct': False, 'second': False, 'street': False, 'even': True, '+': False, 'above': False, 'new': True, 'poorly': False, 'ever': False, 'disney': False, 'told': False, 'hero': False, 'mel': False, 'human': False, 'men': False, 'here': False, 'studio': False, 'cult': False, '100': False, 'kids': True, 'daughter': False, 'leaves': False, 'changed': False, 'credit': False, 'military': False, 'changes': False, 'fantastic': False, 'julie': False, 'explained': False, 'julia': False, 'highly': False, 'brought': False, 'moral': False, 'actions': F

In [29]:
featuresets = [(document_features(d),c) for d,c in documents]
training_set = featuresets[:100]
test_set = featuresets[100:]

In [30]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [31]:
nltk.classify.accuracy(classifier,test_set)

0.7347368421052631

In [32]:
classifier.show_most_informative_features(5)

Most Informative Features
               hilarious = True              pos : neg    =      6.1 : 1.0
                 writers = True              neg : pos    =      5.9 : 1.0
                 leaving = True              pos : neg    =      5.4 : 1.0
                  choice = True              pos : neg    =      5.4 : 1.0
                  superb = True              pos : neg    =      5.4 : 1.0


# Parts of Speech Tagging

In [33]:
from nltk.corpus import brown

In [34]:
suffixes = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffixes.inc(word[-1:])
    suffixes.inc(word[-2:])
    suffixes.inc(word[-3:])

In [35]:
common = suffixes.keys()[:100]
print common

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [36]:
def pos_features(w):
    features = {}
    for s in common:
        features[s] = w.lower().endswith(s)
    return features

In [37]:
tagged_words = brown.tagged_words(categories='news')

In [38]:
featuresets = [(pos_features(w),c) for w,c in tagged_words]
n = len(featuresets)
print n

100554


In [39]:
training_set = featuresets[n//10:]
test_set = featuresets[:n//10]

In [40]:
classifier = nltk.DecisionTreeClassifier.train(training_set)

In [41]:
nltk.classify.accuracy(classifier,test_set)

0.6270512182993535

In [42]:
classifier.classify(pos_features('cats'))

'NNS'

In [43]:
print classifier.pseudocode(depth=4)

if the == False: 
  if , == False: 
    if s == False: 
      if . == False: return '.'
      if . == True: return '.'
    if s == True: 
      if is == False: return 'PP$'
      if is == True: return 'BEZ'
  if , == True: return ','
if the == True: return 'AT'

